In [1]:
'''
1. 将拼多多收入与订单合并
2. 对收入分别按订单汇总、按存货编码汇总
'''

import easygui
import os
import pandas as pd
import re
import numpy as np



In [2]:
#所有订单
fname_dingdan = r"F:\a00nutstore\008\zw08\电商\拼多多\100306881orders_export2024-09-30-16-58-49.csv"
df_dingdan0 = pd.read_csv(fname_dingdan)
df_dingdan0.head()

,商品,订单号,订单状态,商品数量(件),是否审核中,支付时间,承诺发货时间,承诺送达时间,发货时间,确认收货时间,...,预约配送时间,是否节能补贴,是否分期,分期期数,手续费承担方,是否无痕发货,订单来源,是否社区团购,小区送达情况,分期方式
0,500页加厚英语定位练习纸本英语定位纸英文专用纸小学初中高中,240929-658935703962749,已发货，待收货,1,正常,2024/9/29 23:49,2024/10/1 23:49,NaN,2024/9/30 11:14,\t,...,\t,否,否,NaN,\t,否,\t,否,NaN,NaN
1,500页加厚英语定位练习纸本英语定位纸英文专用纸小学初中高中,240929-379762829722749,已发货，待收货,1,正常,2024/9/29 23:49,2024/10/1 23:49,NaN,2024/9/30 11:14,\t,...,\t,否,否,NaN,\t,否,\t,否,NaN,NaN
2,莱特5本加厚200页B5学生笔记本办公记事本白领工作记录本子笔记本,240929-211738969641353,已发货，待收货,1,正常,2024/9/29 23:44,2024/10/1 23:44,NaN,2024/9/30 11:14,\t,...,\t,否,否,NaN,\t,否,\t,否,NaN,NaN
3,莱特5本加厚200页B5学生笔记本办公记事本白领工作记录本子笔记本,240929-611277893212149,已发货，待收货,1,正常,2024/9/29 23:37,2024/10/1 23:37,NaN,2024/9/30 11:14,\t,...,\t,否,否,NaN,\t,否,\t,否,NaN,NaN
4,莱特5本加厚200页B5学生笔记本办公记事本白领工作记录本子笔记本,240929-080425849263824,未发货，退款成功,1,正常,2024/9/29 23:34,2024/10/1 23:34,NaN,\t,\t,...,\t,否,否,NaN,\t,否,\t,否,NaN,NaN


In [ ]:
df_dingdan0.columns.to_list()

In [5]:
dic = {'五': 5,
       '六': 6,
       '七': 7,
       '八': 8,
       '九': 9,
       '十': 10,
       '四': 4,
       '三': 3,
       '二': 2,
       '一': 1}


def addBen(string):
    # string = r'(81本)'
    pattern1 = r'(?P<ben>\d+)本'
    pattern2 = r'(?P<ben>[十九八七六五四三二一])本'
    regexp1 = re.compile(pattern1)
    mat1 = regexp1.search(string)
    regexp2 = re.compile(pattern2)
    mat2 = regexp2.search(string)
    if mat1:
        ben = int(mat1.group('ben'))
    else:
        if mat2:
            ben = dic.get(mat2.group('ben'))
        else:
            ben = 1
    return ben


# df1 = df1[lst]
# df1['含量'] = df1['规格名称'].map(addBen)
# df1 = df1.assign(本数=df1['数量'] * df1['含量'])

In [6]:
df_dingdan1 = df_dingdan0[['订单号','发货时间','商家编码-规格维度','商家实收金额(元)','商品数量(件)','商品规格']]
df_dingdan1.columns = ['dingdanhao','riqi','code','jiner','shuliang','guige']
df_dingdan1[df_dingdan1['dingdanhao'].isnull()]
df_dingdan1['hanliang'] = df_dingdan1['guige'].map(addBen)
df_dingdan1 = df_dingdan1.assign(shuliang=df_dingdan1['shuliang'] * df_dingdan1['hanliang'])
df_dingdan1.head()

C:\Users\redda\AppData\Local\Temp\ipykernel_26092\402509756.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dingdan1['hanliang'] = df_dingdan1['guige'].map(addBen)


,dingdanhao,riqi,code,jiner,shuliang,guige,hanliang
0,240929-658935703962749,2024/9/30 11:14,EJDWB550-英高,7.84,5,"高级版,5本",5
1,240929-379762829722749,2024/9/30 11:14,EJDWB550-英初,7.83,5,"初级版,5本",5
2,240929-211738969641353,2024/9/30 11:14,N0581,7.85,5,B5五本（每本40张）,5
3,240929-611277893212149,2024/9/30 11:14,N0483,9.57,5,A5五本（每本80张）,5
4,240929-080425849263824,\t,N0481,4.27,3,A5三本（每本40张）,3


In [7]:
df_dingdan2 = df_dingdan1.iloc[:,:5]
df_dingdan2 = df_dingdan2.set_index('dingdanhao')
df_dingdan2.head()

,riqi,code,jiner,shuliang
dingdanhao,,,,
240929-658935703962749,2024/9/30 11:14,EJDWB550-英高,7.84,5
240929-379762829722749,2024/9/30 11:14,EJDWB550-英初,7.83,5
240929-211738969641353,2024/9/30 11:14,N0581,7.85,5
240929-611277893212149,2024/9/30 11:14,N0483,9.57,5
240929-080425849263824,\t,N0481,4.27,3


In [34]:
#所有收入
fname_shouru = r"F:\a00nutstore\008\zw08\电商\拼多多\pdd-mall-bill-detail(100306881)_2024-09-30-07-09-49_3.csv"
df_shouru0 = pd.read_csv(fname_shouru,encoding = 'gb18030')
df_shouru0

,商户订单号,发生时间,收入金额（+元）,支出金额（-元）,账务类型,备注,业务描述
0,240928-589247332703567,2024/9/28 23:45,0.07,0.00,技术服务费,基础技术服务费返还,0030002|技术服务费-基础技术服务费
1,240928-589247332703567,2024/9/28 23:45,0.00,-10.48,退款,-,0020002|交易退款-订单退款
2,240928-589247332703567,2024/9/28 23:45,0.00,-1.82,优惠券结算,退款扣除优惠券支付金额,0020005|交易退款-优惠券结算
3,240928-589247332703567,2024/9/28 23:45,10.48,0.00,交易收入,-,0010002|交易收入-订单收入
4,240928-589247332703567,2024/9/28 23:45,1.82,0.00,优惠券结算,交易成功优惠券支付金额结算,0010005|交易收入-优惠券结算
...,...,...,...,...,...,...,...
64699,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64700,#支出合计：38362笔-271037.53元,NaN,NaN,NaN,NaN,NaN,NaN
64701,#收入合计：26337笔245178.31元,NaN,NaN,NaN,NaN,NaN,NaN
64702,#总计：-25859.22元,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df_shouru1 = df_shouru0[['商户订单号','发生时间','收入金额（+元）','支出金额（-元）','账务类型','备注','业务描述']]
df_shouru1 = df_shouru1.dropna(thresh = 3)   #至少有三个缺失值 
# df_shouru1['商户订单号'] = df_shouru1['商户订单号'].fillna('888888-888888888888888')

In [36]:

regax = r'（订单(?P<dingdanhao>\d{6}-\d{15})）'
pattern = re.compile(regax)
# mat = pattern.search(string)
def getDingdanhao(s):
    mat = pattern.search(s)
    if mat :
        s = mat.group('dingdanhao')
    else :
        s = ''
    return s

In [ ]:
# string = '2024-09-28售后补偿现金券扣款（订单240923-652476454442206）'


# pattern = re.compile(regax)
# mat = pattern.search(string)
# if mat :
#     dingdanhao = mat.group('dingdanhao')
# else :
#     pass
# dingdanhao

In [ ]:
df_shouru1['商户订单号'].to_excel('商户订单号.xlsx',index = False)

In [37]:
df_shouru1['商户订单号']= df_shouru1['商户订单号'].mask(df_shouru1['商户订单号'].isnull(),df_shouru1['备注'].map(getDingdanhao))



In [40]:
df_shouru1['商户订单号'] = df_shouru1['商户订单号'].replace('','888888-888888888888888')

df_shouru1[df_shouru1['商户订单号'] =='888888-888888888888888']

,商户订单号,发生时间,收入金额（+元）,支出金额（-元）,账务类型,备注,业务描述
526,888888-888888888888888,2024/9/28 13:31,0.00,-10000.0,提现,提现申请,0080001|提现-提现申请
900,888888-888888888888888,2024/9/27 15:19,0.06,0.0,其他,技术服务费返还，批次订单共计2单，共计汇入金额0.06元，关联批次号为BO240926038...,0030001|技术服务费-技术服务费
922,888888-888888888888888,2024/9/27 14:15,0.05,0.0,其他,技术服务费返还，批次订单共计2单，共计汇入金额0.05元，关联批次号为BO240925038...,0030001|技术服务费-技术服务费
1632,888888-888888888888888,2024/9/26 19:35,0.00,-5.0,扣款,商家物流服务承诺未兑现（缺货）,0040006|售后费用-缺货
1695,888888-888888888888888,2024/9/26 17:34,0.00,-15.0,扣款,商家物流服务承诺未兑现（缺货）,0040006|售后费用-缺货
...,...,...,...,...,...,...,...
58692,888888-888888888888888,2024/9/2 23:23,0.00,-156.0,扣款,物流服务承诺未兑现（延迟发货）扣款，数量：52张，金额156元。,0040004|售后费用-延迟发货
58963,888888-888888888888888,2024/9/2 21:14,0.00,-10.0,扣款,商家物流服务承诺未兑现（虚假发货）扣款，违规编号：240901030222133480,0040005|售后费用-虚假发货
59966,888888-888888888888888,2024/9/2 15:20,0.00,-25000.0,提现,提现申请,0080001|提现-提现申请
61637,888888-888888888888888,2024/9/1 19:39,0.00,-45.0,扣款,商家物流服务承诺未兑现（缺货）,0040006|售后费用-缺货


In [41]:
df_shouru1.columns = ['dingdanhao', 'riqi', 'ru', 'chu', 'leixing', 'beizhu', 'miaoshu']
df_shouru2 = df_shouru1.loc[df_shouru1.leixing != '提现']
df_shouru2 = df_shouru2.assign(shouru = df_shouru2['ru'] - df_shouru2['chu'])
# df_shouru2['dingdanhao'] = df_shouru2['dingdanhao'].fillna('888888-888888888888888')
shouru_pivot =df_shouru2.pivot_table(index = 'dingdanhao',values =['ru','chu','shouru'],aggfunc = np.sum)
shouru_pivot

C:\Users\redda\AppData\Local\Temp\ipykernel_26092\1742693002.py:5: FutureWarning: The provided callable <function sum at 0x0000000008A21760> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  shouru_pivot =df_shouru2.pivot_table(index = 'dingdanhao',values =['ru','chu','shouru'],aggfunc = np.sum)


,chu,ru,shouru
dingdanhao,,,
240625-246792907173009,-9.30,0.00,9.30
240726-232312041021446,-10.50,0.00,10.50
240806-480866517431573,-3.00,0.00,3.00
240809-164878108243433,-3.03,4.62,7.65
240809-238215523923433,-3.03,5.52,8.55
...,...,...,...
240928-655968212500341,-4.37,4.37,8.74
240928-657457201572709,-0.16,0.00,0.16
240928-676488886633923,-0.16,0.00,0.16


In [42]:
result0 =pd.merge(shouru_pivot,df_dingdan2,left_index = True,right_index = True,how = 'outer')
result0

,chu,ru,shouru,riqi,code,jiner,shuliang
dingdanhao,,,,,,,
240625-246792907173009,-9.30,0.00,9.3,NaN,NaN,NaN,NaN
240702-000068165411457,NaN,NaN,NaN,2024/7/2 12:17,N0781(线条),5.90,5.0
240702-002668706081035,NaN,NaN,NaN,2024/7/2 8:15,N0781(方格),5.85,5.0
240702-012284137890533,NaN,NaN,NaN,2024/7/3 8:07,N0482,8.10,5.0
240702-016968630670521,NaN,NaN,NaN,2024/7/3 8:07,N0783,10.30,10.0
...,...,...,...,...,...,...,...
240929-682077734432425,NaN,NaN,NaN,2024/9/29 15:43,RFZW-A460N（500格）,15.50,5.0
240929-683021473293766,NaN,NaN,NaN,2024/9/29 11:48,N0481,4.27,3.0
240929-684940367432513,NaN,NaN,NaN,2024/9/29 11:48,FYP1660,13.50,5.0


In [43]:
#空值填充为0
for i in ['chu','ru','shouru','jiner','shuliang']:
    result0[i] = result0[i].fillna(0)
#code填充
result0['code'] = result0['code'].fillna('8888')

#日期填充
result0['riqi'] = result0['riqi'].replace('\t',None)
result0['riqi'] = result0['riqi'].ffill()
result0['riqi'] = result0['riqi'].bfill()
result0

,chu,ru,shouru,riqi,code,jiner,shuliang
dingdanhao,,,,,,,
240625-246792907173009,-9.30,0.00,9.3,2024/7/2 12:17,8888,0.00,0.0
240702-000068165411457,0.00,0.00,0.0,2024/7/2 12:17,N0781(线条),5.90,5.0
240702-002668706081035,0.00,0.00,0.0,2024/7/2 8:15,N0781(方格),5.85,5.0
240702-012284137890533,0.00,0.00,0.0,2024/7/3 8:07,N0482,8.10,5.0
240702-016968630670521,0.00,0.00,0.0,2024/7/3 8:07,N0783,10.30,10.0
...,...,...,...,...,...,...,...
240929-682077734432425,0.00,0.00,0.0,2024/9/29 15:43,RFZW-A460N（500格）,15.50,5.0
240929-683021473293766,0.00,0.00,0.0,2024/9/29 11:48,N0481,4.27,3.0
240929-684940367432513,0.00,0.00,0.0,2024/9/29 11:48,FYP1660,13.50,5.0


In [44]:
result0.to_excel('分配前outer-2-6.xlsx')

C:\Users\redda\AppData\Local\Temp\ipykernel_26092\2364586390.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  result0.to_excel('分配前outer-2-6.xlsx')


In [45]:
df_dingdan2.to_excel('订单号-6.xlsx')

C:\Users\redda\AppData\Local\Temp\ipykernel_26092\3661604964.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df_dingdan2.to_excel('订单号-6.xlsx')


In [ ]:
# left = pd.read_clipboard()
# right = pd.read_clipboard()
# test_result = pd.merge(left = left,right = right,how = 'outer')
# test_result = pd.merge(right = right,left = left,how = 'outer')
# test_result 

In [46]:
gp = result0.groupby('dingdanhao')
gp

In [18]:
#将订单总金额按各产品的明细销售额进行分配
data = []
for k,v in gp:
    # print(k,v)
    mean = v.shouru.mean()
    total = v['jiner'].sum()
    # print('mean',mean)
    # print('total',total)
    v['result_jiner'] = mean/total*(v.jiner)
    data.append(v)
result1 = pd.concat(data)
result1 
    

C:\Users\redda\AppData\Local\Temp\ipykernel_26092\2925582487.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  v['result_jiner'] = mean/total*(v.jiner)
C:\Users\redda\AppData\Local\Temp\ipykernel_26092\2925582487.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  v['result_jiner'] = mean/total*(v.jiner)


,chu,ru,shouru,riqi,code,jiner,shuliang,result_jiner
dingdanhao,,,,,,,,
,-1112.38,18.32,1130.7,2024/7/2 12:17,8888,0.00,0.0,NaN
240625-246792907173009,-9.30,0.00,9.3,2024/7/2 12:17,8888,0.00,0.0,NaN
240702-000068165411457,0.00,0.00,0.0,2024/7/2 12:17,N0781(线条),5.90,5.0,0.0
240702-002668706081035,0.00,0.00,0.0,2024/7/2 8:15,N0781(方格),5.85,5.0,0.0
240702-012284137890533,0.00,0.00,0.0,2024/7/3 8:07,N0482,8.10,5.0,0.0
...,...,...,...,...,...,...,...,...
240929-678994941551326,0.00,0.00,0.0,2024/9/29 15:43,N0582,10.90,5.0,0.0
240929-682077734432425,0.00,0.00,0.0,2024/9/29 15:43,RFZW-A460N（500格）,15.50,5.0,0.0
240929-683021473293766,0.00,0.00,0.0,2024/9/29 11:48,N0481,4.27,3.0,0.0


In [48]:
result1 = result1.rename({'':'888888-888888888888888'})
result1

,chu,ru,shouru,riqi,code,jiner,shuliang,result_jiner
dingdanhao,,,,,,,,
888888-888888888888888,-1112.38,18.32,1130.7,2024/7/2 12:17,8888,0.00,0.0,NaN
240625-246792907173009,-9.30,0.00,9.3,2024/7/2 12:17,8888,0.00,0.0,NaN
240702-000068165411457,0.00,0.00,0.0,2024/7/2 12:17,N0781(线条),5.90,5.0,0.0
240702-002668706081035,0.00,0.00,0.0,2024/7/2 8:15,N0781(方格),5.85,5.0,0.0
240702-012284137890533,0.00,0.00,0.0,2024/7/3 8:07,N0482,8.10,5.0,0.0
...,...,...,...,...,...,...,...,...
240929-678994941551326,0.00,0.00,0.0,2024/9/29 15:43,N0582,10.90,5.0,0.0
240929-682077734432425,0.00,0.00,0.0,2024/9/29 15:43,RFZW-A460N（500格）,15.50,5.0,0.0
240929-683021473293766,0.00,0.00,0.0,2024/9/29 11:48,N0481,4.27,3.0,0.0


In [49]:
result1.to_excel('分配后6.xlsx')

C:\Users\redda\AppData\Local\Temp\ipykernel_26092\3595823324.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  result1.to_excel('分配后6.xlsx')
